In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

In [ ]:
pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

In [ ]:
# Read data from CSV
df = pd.read_csv("https://raw.githubusercontent.com/guebin/DV2023/main/posts/NYCTaxi.csv").assign(
    log_trip_duration=lambda df: np.log(df.trip_duration),
    pickup_datetime=lambda df: df.pickup_datetime.apply(pd.to_datetime),
    dropoff_datetime=lambda df: df.dropoff_datetime.apply(pd.to_datetime),
    dist=lambda df: np.sqrt(
        (df.pickup_latitude - df.dropoff_latitude) ** 2
        + (df.pickup_longitude - df.dropoff_longitude) ** 2
    ),
    # --- #
    vendor_id=lambda df: df.vendor_id.map({1: "A", 2: "B"}),
)

In [ ]:
week_k = {0:'월',1:'화',2:'수',3:'목',4:'금',5:'토',6:'일'}
df["dayofweek"] = df["dayofweek"].map(week_k)

In [ ]:
tidydata = df.pivot_table(
    index = 'pickup_hour',
    columns = 'dayofweek',
    values = 'speed',
    aggfunc = 'mean'
).stack().reset_index().rename({0:'speed_mean'},axis=1)
tidydata

In [ ]:
fig =px.density_heatmap(
    data_frame=tidydata,
    x='pickup_hour',
    y='dayofweek',
    z='speed_mean',
    nbinsx=24,
    nbinsy=7,
    height=300,
    # color_continuous_scale=px.colors.sequential.Inferno,
    #color=dict(title="평균 속도")  # 여기에 원하는 이름을 입력
)
fig.update_layout(
    xaxis=dict(
        title='탑승 시간'
    ),
    yaxis=dict(
        title='요일'
    ),
#     zaxis=dict(
#         title='평균 속도 합'    
#     )
    
)

In [ ]:
tidydata = df.pivot_table(
    index = 'pickup_hour',
    columns = 'dayofweek',
    values = 'dist',
    aggfunc = 'mean'
).stack().reset_index().rename({0:'dist_mean'},axis=1)
px.density_heatmap(
    data_frame=tidydata,
    x='pickup_hour',
    y='dayofweek',
    z='dist_mean',
    nbinsx=24,
    nbinsy=7,
    height=300
)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Plotting configuration
pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

# Read data from CSV
df = pd.read_csv("https://raw.githubusercontent.com/guebin/DV2023/main/posts/NYCTaxi.csv").assign(
    log_trip_duration=lambda df: np.log(df.trip_duration),
    pickup_datetime=lambda df: df.pickup_datetime.apply(pd.to_datetime),
    dropoff_datetime=lambda df: df.dropoff_datetime.apply(pd.to_datetime),
    dist=lambda df: np.sqrt(
        (df.pickup_latitude - df.dropoff_latitude) ** 2
        + (df.pickup_longitude - df.dropoff_longitude) ** 2
    ),
    # --- #
    vendor_id=lambda df: df.vendor_id.map({1: "A", 2: "B"}),
)

# Calculate additional attributes
df = df.assign(
    speed=lambda df: df.dist / df.trip_duration,
    pickup_hour=lambda df: df.pickup_datetime.dt.hour,
    dropoff_hour=lambda df: df.dropoff_datetime.dt.hour,
    dayofweek=lambda df: df.pickup_datetime.dt.dayofweek,
)

# Reduce data size for faster processing
df_small = df[::100].reset_index(drop=True)

# Convert day of week
day_map = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"}
df_small["dayofweek"] = df_small["dayofweek"].map(day_map)

# Calculate average speed
avg_speed_by_day_time = (
    df_small.groupby(["dayofweek", "pickup_hour"])["speed"]
    .mean()
    .reset_index()
)

# Line chart visualization
fig = px.line(
    avg_speed_by_day_time,
    x="pickup_hour",
    y="speed",
    color="dayofweek",
    title="Average Speed by Day of Week and Time of Day",
    labels={
        "pickup_hour": "Pickup Hour",
        "speed": "Average Speed (km/h)",
        "dayofweek": "Day of Week",
    },
)

# Show the dashboard
fig.show()